In [2]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [3]:
path = 'data'
images = []
classNames = []
myList = os.listdir(path)

In [4]:
myList

['modi.jpg', 'pratyusa.jpg', 'trump.jpg']

In [5]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['modi', 'pratyusa', 'trump']


In [6]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [7]:
encodeListKnown = findEncodings(images)
print('Encoding Complete')

Encoding Complete


In [8]:
len(encodeListKnown)

3

In [21]:
# import pickle
# data = {"encodings": encodeListKnown, "names": classNames}
# #use pickle to save data into a file for later use
# f = open("face_enc", "wb")
# f.write(pickle.dumps(data))
# f.close()

In [19]:
# img = cv2.imread('pratyusa.jpg', cv2.IMREAD_COLOR)
cap = cv2.VideoCapture(0)


while True:
    success, img = cap.read()
    
    img = cv2.flip(img,1)
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)


    face_locations = face_recognition.face_locations(imgS)
    face_encodings = face_recognition.face_encodings(imgS, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        name = "Unknown"
        
        matches = face_recognition.compare_faces(encodeListKnown, face_encoding)
        face_distances = face_recognition.face_distance(encodeListKnown, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            name = classNames[best_match_index]

    face_names.append(name)


    for faceLoc in face_locations:
        y1, x2, y2, x1 = faceLoc
        y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)


#     print(face_names)

    cv2.imshow('image',img)
    if cv2.waitKey(1) == 13: #press enter to exist
        break

cap.release()
cv2.destroyAllWindows() 

print(face_names)

['pratyusa']


In [16]:
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%I:%M:%S')
        f.writelines(f'\n{name},{dtString}')

In [18]:
markAttendance(name)